In [2]:
import pandas as pd

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
# Load in previously processed dataset
application_train = pd.read_csv('./home-credit-train.csv')
application_train.shape

(276759, 215)

In [4]:
application_valid = pd.read_csv('./home-credit-valid.csv')
application_valid.shape

(30752, 215)

In [5]:
application = pd.concat([application_train, application_valid], axis=0)

In [6]:
IDS = application['SK_ID_CURR']

In [10]:
rows = 100000

In [7]:
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

""" Every SK_ID_CURR in the application_train.csv dataset is linked 
to several SK_ID_PREV in the other csv files. We are going to group the data 
in the other datasets according to the SK_ID_CURR, and then extract aggregate metrics
for each of these groups

"""


def process_group(group, column, prefix):
    try:
        group[-1].drop(labels=['SK_ID_PREV', 'SK_ID_CURR'], inplace=True, axis=1)
        
    except BaseException as e:
        #edge case with bureau.csv
        group[-1].drop(labels=['SK_ID_CURR', 'SK_ID_BUREAU'], inplace=True, axis=1)
        
    numerical_features = group[-1].select_dtypes(include=['float', 'int']).mean(axis=0).to_frame().T    
    categorical_features = group[-1].select_dtypes(include=['object']).mode()
                 
    features = pd.concat([numerical_features, categorical_features], axis=1)
    features.columns = [prefix + str(col) for col in features.columns]
    features[column] = group[0]
    
    return features

    
def generator(groups, column, prefix):
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_group, group, column, prefix): group for group in groups}
        
        for future in concurrent.futures.as_completed(futures):
            yield future.result()
            
        
def aggregate(df, column, prefix):
    output = pd.DataFrame()
    _df = convert_types(df)
    
    # Group data by SK_ID_CURR
    groups = list(_df.groupby(column))
    gen = generator(groups, column, prefix)
    for result in gen:
        output = output.append(result, ignore_index=True)
        
    return output

In [8]:
import numpy as np

# From:
# https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering-p2
def convert_types(df):  
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    print('Original memory usage (GB):', original_memory / 1e9 )
    print('New memory usage (GB):', new_memory / 1e9 )
            
    return df

In [11]:
prev_app = pd.read_csv('./previous_application.csv',nrows=rows)
prev_app = prev_app.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop=True)
prev_app.shape

(100000, 37)

In [ ]:
prev_app_agg = aggregate(prev_app, 'SK_ID_CURR', 'PREV_APP_')
del prev_app

In [15]:
prev_app_agg.shape

(96939, 36)

In [18]:
credit_balance = pd.read_csv('./credit_card_balance.csv', nrows=rows)
credit_balance = credit_balance.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
credit_balance.shape

(100000, 23)

In [ ]:
credit_balance_agg = aggregate(credit_balance, 'SK_ID_CURR', 'CREDIT_BALANCE_')
del credit_balance

In [21]:
credit_balance_agg.shape

(54464, 22)

In [23]:
credit_balance_agg.to_csv('credit_balance_agg.csv', index=False)

In [22]:
pos_cash = pd.read_csv('./POS_CASH_balance.csv', nrows=rows)
pos_cash = pos_cash.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
pos_cash.shape

(100000, 8)

In [24]:
pos_cash_agg = aggregate(pos_cash, 'SK_ID_CURR', 'POS_CASH_')
del pos_cash

In [28]:
pos_cash_agg.to_csv('pos_cash_agg.csv', index=False)

In [30]:
installments = pd.read_csv('./installments_payments.csv', nrows=rows)
installments = installments.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
installments.shape

(100000, 8)

In [31]:
installments_agg = aggregate(installments, 'SK_ID_CURR', 'INSTALLMENTS_')
del installments

In [32]:
installments_agg.shape

(148590, 7)

In [33]:
bureau = pd.read_csv('./bureau.csv', nrows=rows)
bureau = bureau.sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop=True)
bureau_balance = pd.read_csv('./bureau_balance.csv', nrows=rows)
bureau_balance = bureau_balance.sort_values('SK_ID_BUREAU').reset_index(drop = True)

In [34]:
bureau_merged = pd.merge(bureau, bureau_balance, on=['SK_ID_BUREAU'])

In [35]:
bureau_merged.shape

(80066, 19)

In [36]:
bureau_agg = aggregate(bureau_merged, 'SK_ID_CURR', 'BUREAU_')

In [37]:
bureau_agg.shape

(545, 18)

In [38]:
dataframes = [application, prev_app_agg, installments_agg, pos_cash_agg, credit_balance_agg, bureau_agg]

In [39]:
from functools import reduce

df_merged = reduce(lambda left, right: pd.merge(left,right.drop_duplicates('SK_ID_CURR'), 
                                                on=['SK_ID_CURR'], how='left'), dataframes)

In [40]:
df_merged.shape

(307511, 300)

In [17]:
""" Drop all feature columns with 90% NaNs
"""

THRESHOLD = 0.9

def filterna(df):
    for c in df.columns.tolist():
        ratio = (df[c].isnull().sum() / df.shape[0])
        if ratio > THRESHOLD:
            print(c, ratio)
            df = df.drop(labels=[c], axis=1)
    
    return df

In [42]:
filtered = filterna(df_merged)

PREV_APP_RATE_INTEREST_PRIMARY 0.9991057230473056
PREV_APP_RATE_INTEREST_PRIVILEGED 0.9991057230473056
BUREAU_DAYS_CREDIT 0.9990276770587069
BUREAU_CREDIT_DAY_OVERDUE 0.9990276770587069
BUREAU_DAYS_CREDIT_ENDDATE 0.9990536923882398
BUREAU_DAYS_ENDDATE_FACT 0.9991479979577966
BUREAU_AMT_CREDIT_MAX_OVERDUE 0.9993918916721678
BUREAU_CNT_CREDIT_PROLONG 0.9990276770587069
BUREAU_AMT_CREDIT_SUM 0.9990276770587069
BUREAU_AMT_CREDIT_SUM_DEBT 0.999060196220623
BUREAU_AMT_CREDIT_SUM_LIMIT 0.9991252345444553
BUREAU_AMT_CREDIT_SUM_OVERDUE 0.9990276770587069
BUREAU_DAYS_CREDIT_UPDATE 0.9990276770587069
BUREAU_AMT_ANNUITY 0.9991577537063715
BUREAU_MONTHS_BALANCE 0.9990276770587069
BUREAU_CREDIT_ACTIVE 0.9990276770587069
BUREAU_CREDIT_CURRENCY 0.9990276770587069
BUREAU_CREDIT_TYPE 0.9990276770587069
BUREAU_STATUS 0.9990276770587069


In [44]:
filtered.shape

(307511, 281)

In [12]:
filtered = pd.read_csv('home-credit-all-features.csv')

In [ ]:
filtered.fillna('ffill',  inplace=True)
filtered.fillna('bfill', inplace=True)

In [25]:
categorical_features = filtered.select_dtypes(include=['object'])
categorical_features.shape

(307511, 18)

In [26]:
categorical_features.columns

Index(['PREV_APP_NAME_CONTRACT_TYPE', 'PREV_APP_WEEKDAY_APPR_PROCESS_START',
       'PREV_APP_FLAG_LAST_APPL_PER_CONTRACT',
       'PREV_APP_NAME_CASH_LOAN_PURPOSE', 'PREV_APP_NAME_CONTRACT_STATUS',
       'PREV_APP_NAME_PAYMENT_TYPE', 'PREV_APP_CODE_REJECT_REASON',
       'PREV_APP_NAME_TYPE_SUITE', 'PREV_APP_NAME_CLIENT_TYPE',
       'PREV_APP_NAME_GOODS_CATEGORY', 'PREV_APP_NAME_PORTFOLIO',
       'PREV_APP_NAME_PRODUCT_TYPE', 'PREV_APP_CHANNEL_TYPE',
       'PREV_APP_NAME_SELLER_INDUSTRY', 'PREV_APP_NAME_YIELD_GROUP',
       'PREV_APP_PRODUCT_COMBINATION', 'POS_CASH_NAME_CONTRACT_STATUS',
       'CREDIT_BALANCE_NAME_CONTRACT_STATUS'],
      dtype='object')

In [27]:
encoded_categorical_features = pd.get_dummies(categorical_features)

In [28]:
encoded_categorical_features.shape

(307511, 151)

In [13]:
numerical_features = filtered.select_dtypes(exclude=['category', 'object'])

In [47]:
numerical_features.select_dtypes(exclude='object').to_csv('home-credit-numerical-features.csv', index=False)

In [ ]:
! python ../manual_features.py --data ./home-credit-numerical-features.csv --outfile home-credit-numerical-processed.csv

In [30]:
processed_numerical_features = pd.read_csv('./home-credit-numerical-processed.csv')

In [31]:
final_features = pd.concat([processed_numerical_features, categorical_features], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(final_features.drop('TARGET', axis=1), final_features['TARGET'], 
                                                    test_size=0.2, random_state=42)

In [35]:
train = pd.concat([X_train, y_train], axis=1)
valid = pd.concat([X_valid, y_valid], axis=1)

In [42]:
train.drop('Unnamed: 0', axis=1, inplace=True)
train.select_dtypes(exclude=['object']).to_csv('home-credit-manual-train.csv')

In [43]:
valid.drop('Unnamed: 0', axis=1, inplace=True)
valid.select_dtypes(exclude=['object']).to_csv('home-credit-manual-valid.csv')